In [58]:
from flask import Flask, Response
from typing import List, Optional
import mysql.connector
from pydantic import BaseModel, Field
import json
import datetime as dt

app = Flask(__name__)


class TradeDetails(BaseModel):
    buySellIndicator: str = Field(..., description="A value of BUY for buys, SELL for sells.")
    price: float = Field(..., description="The price of the Trade.")
    quantity: int = Field(..., description="The amount of units traded.")


class Trade(BaseModel):
    asset_class: Optional[str] = Field(alias="asset_class", description="The asset class of the instrument traded. E.g. Bond, Equity, FX...etc")
    counterparty: Optional[str] = Field(default=None, description="The counterparty the trade was executed with. May not always be available")
    instrument_id: str = Field(alias="instrument_id", description="The ISIN/ID of the instrument traded. E.g. TSLA, AAPL, AMZN...etc")
    instrument_name: str = Field(alias="instrument_name", description="The name of the instrument traded.")
    trade_date_time: dt.datetime = Field(alias="trade_date_time", description="The date-time the Trade was executed")
    trade_details: TradeDetails = Field(alias="trade_details", description="The details of the trade, i.e. price, quantity")
    trade_id: str = Field(alias="trade_id", description="The unique ID of the trade")
    trader: str = Field(description="The name of the Trader")


def fetch_trades_from_db():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades")
        trades = []
        for trade_data in cursor.fetchall():
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            trades.append(trade.dict())
        cursor.close()
        connection.close()
        return trades
    except mysql.connector.Error as error:
        # Handle the error
        print(f"Error connecting to MySQL: {error}")
        return []


def fetch_trade_by_id(trade_id):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades WHERE trade_id = %s", (trade_id,))
        trade_data = cursor.fetchone()
        if trade_data:
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            return trade.dict()
        else:
            return None
        cursor.close()
        connection.close()
    except mysql.connector.Error as error:
        # Handle the error
        print(f"Error connecting to MySQL: {error}")
        return None


class HomePage:
    @app.route("/", methods=["GET"])
    def home():
        return "Welcome to the Trades API"


class TradesPage:
    @app.route("/trades", methods=["GET"])
    @app.route("/trades/<trade_id>", methods=["GET"])
    def get_trades(trade_id=None):
        if trade_id:
            trade = fetch_trade_by_id(trade_id)
            if trade:
                response = json.dumps(trade, indent=4, default=str)  # Beautify JSON response
                return Response(response, content_type="application/json")
            else:
                return Response("Trade not found", status=404)
        else:
            try:
                trades = fetch_trades_from_db()
                response = json.dumps(trades, indent=4, default=str)  # Beautify JSON response
                return Response(response, content_type="application/json")
            except Exception as e:
                # Handle the error
                print(f"Error fetching trades from database: {e}")
                return Response("An error occurred", status=500)


if __name__ == "__main__":
    HomePage()
    TradesPage()
    app.run(host="0.0.0.0", port=8000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.43.79:8000/ (Press CTRL+C to quit)
192.168.43.79 - - [06/Jun/2023 16:15:29] "GET / HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:15:32] "GET /trades HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:15:41] "GET /trades/TRADE003 HTTP/1.1" 200 -


In [59]:
from flask import Flask, Response, request
from typing import List, Optional
import mysql.connector
from pydantic import BaseModel, Field
import json
import datetime as dt

app = Flask(__name__)


class TradeDetails(BaseModel):
    buySellIndicator: str = Field(..., description="A value of BUY for buys, SELL for sells.")
    price: float = Field(..., description="The price of the Trade.")
    quantity: int = Field(..., description="The amount of units traded.")


class Trade(BaseModel):
    asset_class: Optional[str] = Field(alias="asset_class", description="The asset class of the instrument traded. E.g. Bond, Equity, FX...etc")
    counterparty: Optional[str] = Field(default=None, description="The counterparty the trade was executed with. May not always be available")
    instrument_id: str = Field(alias="instrument_id", description="The ISIN/ID of the instrument traded. E.g. TSLA, AAPL, AMZN...etc")
    instrument_name: str = Field(alias="instrument_name", description="The name of the instrument traded.")
    trade_date_time: dt.datetime = Field(alias="trade_date_time", description="The date-time the Trade was executed")
    trade_details: TradeDetails = Field(alias="trade_details", description="The details of the trade, i.e. price, quantity")
    trade_id: str = Field(alias="trade_id", description="The unique ID of the trade")
    trader: str = Field(description="The name of the Trader")


def fetch_trades_from_db():
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades")
        trades = []
        for trade_data in cursor.fetchall():
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            trades.append(trade.dict())
        cursor.close()
        connection.close()
        return trades
    except mysql.connector.Error as error:
        # Handle the error
        print(f"Error connecting to MySQL: {error}")
        return []


def fetch_trade_by_id(trade_id):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades WHERE trade_id = %s", (trade_id,))
        trade_data = cursor.fetchone()
        if trade_data:
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            return trade.dict()
        else:
            return None
        cursor.close()
        connection.close()
    except mysql.connector.Error as error:
        # Handle the error
        print(f"Error connecting to MySQL: {error}")
        return None


class HomePage:
    @app.route("/", methods=["GET"])
    def home():
        return "Welcome to the Trades API"


class TradesPage:
    @app.route("/trades", methods=["GET"])
    @app.route("/trades/<trade_id>", methods=["GET"])
    def get_trades(trade_id=None):
        search_query = request.args.get("search")

        if trade_id:
            trade = fetch_trade_by_id(trade_id)
            if trade:
                response = json.dumps(trade, indent=4, default=str)  # Beautify JSON response
                return Response(response, content_type="application/json")
            else:
                return Response("Trade not found", status=404)
        else:
            try:
                trades = fetch_trades_from_db()
                if search_query:
                    trades = filter_trades_by_search(trades, search_query)
                response = json.dumps(trades, indent=4, default=str)  # Beautify JSON response
                return Response(response, content_type="application/json")
            except Exception as e:
                # Handle the error
                print(f"Error fetching trades from database: {e}")
                return Response("An error occurred", status=500)


def filter_trades_by_search(trades, search_query):
    filtered_trades = []
    for trade in trades:
        if (
            search_query.lower() in trade.get("counterparty", "").lower()
            or search_query.lower() in trade.get("instrument_id", "").lower()
            or search_query.lower() in trade.get("instrument_name", "").lower()
            or search_query.lower() in trade.get("trader", "").lower()
        ):
            filtered_trades.append(trade)
    return filtered_trades


if __name__ == "__main__":
    HomePage()
    TradesPage()
    app.run(host="0.0.0.0", port=8000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.43.79:8000/ (Press CTRL+C to quit)
192.168.43.79 - - [06/Jun/2023 16:26:20] "GET / HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:26:25] "GET /trades HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:26:31] "GET /trades/TRADE003 HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:27:17] "GET /trades/?search=bob%20smith HTTP/1.1" 404 -
192.168.43.79 - - [06/Jun/2023 16:28:24] "GET /trades/aapl HTTP/1.1" 404 -
192.168.43.79 - - [06/Jun/2023 16:28:41] "GET /trades/ HTTP/1.1" 404 -
192.168.43.79 - - [06/Jun/2023 16:28:51] "GET /trades/aapl HTTP/1.1" 404 -
192.168.43.79 - - [06/Jun/2023 16:29:08] "GET / HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:29:11] "GET /trades HTTP/1.1" 200 -
192.168.43.79 - - [06/Jun/2023 16:29:23] "GET /trades?search=bond HTTP/1.1" 200 -


In [62]:
from flask import Flask, Response, request
from typing import List, Optional
import mysql.connector
from pydantic import BaseModel, Field
import json
import datetime as dt

app = Flask(__name__)


class TradeDetails(BaseModel):
    buySellIndicator: str = Field(..., description="A value of BUY for buys, SELL for sells.")
    price: float = Field(..., description="The price of the Trade.")
    quantity: int = Field(..., description="The amount of units traded.")


class Trade(BaseModel):
    asset_class: Optional[str] = Field(alias="asset_class", description="The asset class of the instrument traded. E.g. Bond, Equity, FX...etc")
    counterparty: Optional[str] = Field(default=None, description="The counterparty the trade was executed with. May not always be available")
    instrument_id: str = Field(alias="instrument_id", description="The ISIN/ID of the instrument traded. E.g. TSLA, AAPL, AMZN...etc")
    instrument_name: str = Field(alias="instrument_name", description="The name of the instrument traded.")
    trade_date_time: dt.datetime = Field(alias="trade_date_time", description="The date-time the Trade was executed")
    trade_details: TradeDetails = Field(alias="trade_details", description="The details of the trade, i.e. price, quantity")
    trade_id: str = Field(alias="trade_id", description="The unique ID of the trade")
    trader: str = Field(description="The name of the Trader")


def fetch_trades_from_db(asset_class: Optional[str] = None, start: Optional[dt.datetime] = None, end: Optional[dt.datetime] = None,
                         min_price: Optional[float] = None, max_price: Optional[float] = None, trade_type: Optional[str] = None):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()

        query = "SELECT * FROM trades WHERE 1=1"
        params = []

        if asset_class:
            query += " AND asset_class = %s"
            params.append(asset_class)
        if start:
            query += " AND trade_date_time >= %s"
            params.append(start)
        if end:
            query += " AND trade_date_time <= %s"
            params.append(end)
        if min_price:
            query += " AND trade_details.price >= %s"
            params.append(min_price)
        if max_price:
            query += " AND trade_details.price <= %s"
            params.append(max_price)
        if trade_type:
            query += " AND trade_details.buySellIndicator = %s"
            params.append(trade_type)

        cursor.execute(query, tuple(params))

        trades = []
        for trade_data in cursor.fetchall():
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            trades.append(trade.dict())

        cursor.close()
        connection.close()
        return trades
    except mysql.connector.Error as error:
        print(f"Error connecting to MySQL: {error}")
        return []


def fetch_trade_by_id(trade_id):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades WHERE trade_id = %s", (trade_id,))
        trade_data = cursor.fetchone()
        if trade_data:
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            return trade.dict()
        else:
            return None
        cursor.close()
        connection.close()
    except mysql.connector.Error as error:
        print(f"Error connecting to MySQL: {error}")
        return None


class HomePage:
    @app.route("/", methods=["GET"])
    def home():
        return "Welcome to the Trades API"


class TradesPage:
    @app.route("/trades", methods=["GET"])
    @app.route("/trades/<trade_id>", methods=["GET"])
    def get_trades(trade_id=None):
        search_query = request.args.get("search")
        asset_class = request.args.get("asset_class")
        start = request.args.get("start")
        end = request.args.get("end")
        min_price = request.args.get("min_price")
        max_price = request.args.get("max_price")
        trade_type = request.args.get("trade_type")

        if trade_id:
            trade = fetch_trade_by_id(trade_id)
            if trade:
                response = json.dumps(trade, indent=4, default=str)
                return Response(response, content_type="application/json")
            else:
                return Response("Trade not found", status=404)
        else:
            try:
                trades = fetch_trades_from_db(asset_class=asset_class, start=start, end=end,
                                              min_price=min_price, max_price=max_price, trade_type=trade_type)
                if search_query:
                    trades = filter_trades_by_search(trades, search_query)
                response = json.dumps(trades, indent=4, default=str)
                return Response(response, content_type="application/json")
            except Exception as e:
                print(f"Error fetching trades from database: {e}")
                return Response("An error occurred", status=500)


def filter_trades_by_search(trades, search_query):
    filtered_trades = []
    for trade in trades:
        if (
            search_query.lower() in trade.get("counterparty", "").lower()
            or search_query.lower() in trade.get("instrument_id", "").lower()
            or search_query.lower() in trade.get("instrument_name", "").lower()
            or search_query.lower() in trade.get("trader", "").lower()
        ):
            filtered_trades.append(trade)
    return filtered_trades


if __name__ == "__main__":
    HomePage()
    TradesPage()
    app.run(host="0.0.0.0", port=8000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.106:8000/ (Press CTRL+C to quit)
192.168.0.106 - - [06/Jun/2023 20:55:05] "GET / HTTP/1.1" 200 -
192.168.0.106 - - [06/Jun/2023 20:55:08] "GET /trades?assetClass=Bond&minPrice=500&maxPrice=1000 HTTP/1.1" 200 -
192.168.0.106 - - [06/Jun/2023 20:56:18] "GET /trades?assetClass=Bond&minPrice=500&maxPrice=1000 HTTP/1.1" 200 -


In [73]:
from flask import Flask, Response, request
from typing import List, Optional
import mysql.connector
from pydantic import BaseModel, Field
import json
import datetime as dt

app = Flask(__name__)


class TradeDetails(BaseModel):
    buySellIndicator: str = Field(..., description="A value of BUY for buys, SELL for sells.")
    price: float = Field(..., description="The price of the Trade.")
    quantity: int = Field(..., description="The amount of units traded.")


class Trade(BaseModel):
    asset_class: Optional[str] = Field(alias="asset_class", description="The asset class of the instrument traded. E.g. Bond, Equity, FX...etc")
    counterparty: Optional[str] = Field(default=None, description="The counterparty the trade was executed with. May not always be available")
    instrument_id: str = Field(alias="instrument_id", description="The ISIN/ID of the instrument traded. E.g. TSLA, AAPL, AMZN...etc")
    instrument_name: str = Field(alias="instrument_name", description="The name of the instrument traded.")
    trade_date_time: dt.datetime = Field(alias="trade_date_time", description="The date-time the Trade was executed")
    trade_details: TradeDetails = Field(alias="trade_details", description="The details of the trade, i.e. price, quantity")
    trade_id: str = Field(alias="trade_id", description="The unique ID of the trade")
    trader: str = Field(description="The name of the Trader")


def fetch_trades_from_db(asset_class: Optional[str] = None, start: Optional[dt.datetime] = None, end: Optional[dt.datetime] = None,
                         min_price: Optional[float] = None, max_price: Optional[float] = None, trade_type: Optional[str] = None,
                         page: Optional[int] = None, page_size: Optional[int] = None, sort_by: Optional[str] = None,
                         sort_order: Optional[str] = None):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()

        query = "SELECT * FROM trades WHERE 1=1"
        params = []

        if asset_class:
            query += " AND asset_class = %s"
            params.append(asset_class)
        if start:
            query += " AND trade_date_time >= %s"
            params.append(start)
        if end:
            query += " AND trade_date_time <= %s"
            params.append(end)
        if min_price:
            query += " AND trade_details.price >= %s"
            params.append(min_price)
        if max_price:
            query += " AND trade_details.price <= %s"
            params.append(max_price)
        if trade_type:
            query += " AND trade_details.buySellIndicator = %s"
            params.append(trade_type)

        if sort_by:
            sort_column = "trade_date_time" if sort_by == "trade_date_time" else "trade_details.price"
            sort_order = sort_order or "asc"
            query += f" ORDER BY {sort_column} {sort_order.upper()}"

        if page is not None and page_size is not None:
            offset = page * page_size
            query += f" LIMIT {page_size} OFFSET {offset}"

        cursor.execute(query, tuple(params))

        trades = []
        for trade_data in cursor.fetchall():
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            trades.append(trade.dict())

        cursor.close()
        connection.close()
        return trades
    except mysql.connector.Error as error:
        print(f"Error connecting to MySQL: {error}")
        return []


def fetch_trade_by_id(trade_id):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="mysql@2310",
            database="assignment"
        )
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM trades WHERE trade_id = %s", (trade_id,))
        trade_data = cursor.fetchone()
        if trade_data:
            trade = Trade(
                asset_class=trade_data[0],
                counterparty=trade_data[1],
                instrument_id=trade_data[2],
                instrument_name=trade_data[3],
                trade_date_time=trade_data[4],
                trade_details=TradeDetails(
                    buySellIndicator=trade_data[5],
                    price=float(trade_data[6]),
                    quantity=trade_data[7]
                ),
                trade_id=trade_data[8],
                trader=trade_data[9],
            )
            return trade.dict()
        else:
            return None
        cursor.close()
        connection.close()
    except mysql.connector.Error as error:
        print(f"Error connecting to MySQL: {error}")
        return None


class HomePage:
        @app.route("/", methods=["GET"])
        def home():
            introduction = """
        Welcome to the Trades API!

        Here are the available endpoints:

        1. Retrieve all trades:
           http://192.168.0.106:8000/trades

        2. Retrieve a trade by ID:
           http://192.168.0.106:8000/trades/<trade_id>

        3. Search trades:
           http://192.168.0.106:8000/trades/search?query=<search_query>

        4. Filter trades:
           http://192.168.0.106:8000/trades/filter?asset_class=<asset_class>&start=<start_date>&end=<end_date>&min_price=<min_price>&max_price=<max_price>&trade_type=<trade_type>&page=<page>&page_size=<page_size>&sort_by=<sort_by>&sort_order=<sort_order>

        Note: Replace <trade_id>, <search_query>, <asset_class>, <start_date>, <end_date>, <min_price>, <max_price>, <trade_type>, <page>, <page_size>, <sort_by>, and <sort_order> with the desired values.

        Examples:

        - Retrieve all trades:
          http://192.168.0.106:8000/trades

        - Retrieve a trade by ID:
          http://192.168.0.106:8000/trades/12345

        - Search trades:
          http://192.168.0.106:8000/trades/search?query=Apple

        - Filter trades:
          http://192.168.0.106:8000/trades/filter?asset_class=Equity&start=2023-01-01&end=2023-06-01&min_price=100&max_price=500&trade_type=BUY&page=1&page_size=10&sort_by=trade_date_time&sort_order=desc

        Feel free to explore the API and retrieve trade data based on your requirements.
        """
            return Response(introduction, content_type="text/plain")



class TradesPage:
    @app.route("/trades", methods=["GET"])
    @app.route("/trades/<trade_id>", methods=["GET"])
    def get_trades(trade_id=None):
        search_query = request.args.get("search")
        asset_class = request.args.get("asset_class")
        start = request.args.get("start")
        end = request.args.get("end")
        min_price = request.args.get("min_price")
        max_price = request.args.get("max_price")
        trade_type = request.args.get("trade_type")
        page = int(request.args.get("page", "0"))
        page_size = int(request.args.get("page_size", "10"))
        sort_by = request.args.get("sort_by")
        sort_order = request.args.get("sort_order")

        if trade_id:
            trade = fetch_trade_by_id(trade_id)
            if trade:
                response = json.dumps(trade, indent=4, default=str)
                return Response(response, content_type="application/json")
            else:
                return Response("Trade not found", status=404)
        else:
            try:
                trades = fetch_trades_from_db(asset_class=asset_class, start=start, end=end,
                                              min_price=min_price, max_price=max_price, trade_type=trade_type,
                                              page=page, page_size=page_size, sort_by=sort_by, sort_order=sort_order)
                if search_query:
                    trades = filter_trades_by_search(trades, search_query)
                response = json.dumps(trades, indent=4, default=str)
                return Response(response, content_type="application/json")
            except Exception as e:
                print(f"Error fetching trades from database: {e}")
                return Response("An error occurred", status=500)


def filter_trades_by_search(trades, search_query):
    filtered_trades = []
    for trade in trades:
        if (
            search_query.lower() in trade.get("counterparty", "").lower()
            or search_query.lower() in trade.get("instrument_id", "").lower()
            or search_query.lower() in trade.get("instrument_name", "").lower()
            or search_query.lower() in trade.get("trader", "").lower()
        ):
            filtered_trades.append(trade)
    return filtered_trades


if __name__ == "__main__":
    HomePage()
    TradesPage()
    app.run(host="0.0.0.0", port=8000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.106:8000/ (Press CTRL+C to quit)
192.168.0.106 - - [07/Jun/2023 00:57:59] "GET / HTTP/1.1" 200 -
